## Transformer

In [ ]:
import copy
from typing import Optional, List

import torch
import torch.nn.functional as F
from torch import nn, Tensor


class Transformer(nn.Module):

    def __init__(self, d_model=512, nhead=8, num_encoder_layers=6,
                 num_decoder_layers=6, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False,
                 return_intermediate_dec=False):
        super().__init__()

        encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward,
                                                dropout, activation, normalize_before)
        encoder_norm = nn.LayerNorm(d_model) if normalize_before else None
        self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)

        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward,
                                                dropout, activation, normalize_before)
        decoder_norm = nn.LayerNorm(d_model)
        self.decoder = TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm,
                                          return_intermediate=return_intermediate_dec)
        self.output_layer=nn.Linear(dim_feedforward,4) #num_classes=4

        self._reset_parameters()

        self.d_model = d_model
        self.nhead = nhead

    def _reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, src, mask, query_embed, pos_embed=None):
        # flatten NxCxHxW to HWxNxC
        bs, h, w = src.shape
        # src = src.flatten(1).permute(2, 0, 1)
        src=src.view(-1,1,3)
        # pos_embed = pos_embed.flatten(2).permute(2, 0, 1)
        # query_embed = query_embed.unsqueeze(1).repeat(1, bs, 1)
        # mask = mask.flatten(1)

        tgt = torch.zeros(1,3)
        # tgt = torch.zeros_like(query_embed)
        memory = self.encoder(src, src_key_padding_mask=mask, pos=pos_embed)
        hs = self.decoder(tgt, memory, memory_key_padding_mask=mask,
                          pos=pos_embed, query_pos=query_embed)
        output=self.output_layer(hs)
        # return hs.transpose(1, 2), memory.permute(1, 2, 0).view(bs, c, h, w)
        return nn.Softmax(output)


class TransformerEncoder(nn.Module):

    def __init__(self, encoder_layer, num_layers, norm=None):
        super().__init__()
        self.layers = _get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    def forward(self, src,
                mask: Optional[Tensor] = None,
                src_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None):
        output = src

        for layer in self.layers:
            output = layer(output, src_mask=mask,
                           src_key_padding_mask=src_key_padding_mask, pos=pos)

        if self.norm is not None:
            output = self.norm(output)

        return output


class TransformerDecoder(nn.Module):

    def __init__(self, decoder_layer, num_layers, norm=None, return_intermediate=False):
        super().__init__()
        self.layers = _get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm
        self.return_intermediate = return_intermediate

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None):
        output = tgt

        intermediate = []

        for layer in self.layers:
            output = layer(output, memory, tgt_mask=tgt_mask,
                           memory_mask=memory_mask,
                           tgt_key_padding_mask=tgt_key_padding_mask,
                           memory_key_padding_mask=memory_key_padding_mask,
                           pos=pos, query_pos=query_pos)
            if self.return_intermediate:
                intermediate.append(self.norm(output))

        if self.norm is not None:
            output = self.norm(output)
            if self.return_intermediate:
                intermediate.pop()
                intermediate.append(output)

        if self.return_intermediate:
            return torch.stack(intermediate)

        return output.unsqueeze(0)


class TransformerEncoderLayer(nn.Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
        self.normalize_before = normalize_before

    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    def forward_post(self,
                     src,
                     src_mask: Optional[Tensor] = None,
                     src_key_padding_mask: Optional[Tensor] = None,
                     pos: Optional[Tensor] = None):
        q = k = self.with_pos_embed(src, pos)
        src2 = self.self_attn(q, k, value=src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

    def forward_pre(self, src,
                    src_mask: Optional[Tensor] = None,
                    src_key_padding_mask: Optional[Tensor] = None,
                    pos: Optional[Tensor] = None):
        src2 = self.norm1(src)
        q = k = self.with_pos_embed(src2, pos)
        src2 = self.self_attn(q, k, value=src2, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src2 = self.norm2(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src2))))
        src = src + self.dropout2(src2)
        return src

    def forward(self, src,
                src_mask: Optional[Tensor] = None,
                src_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None):
        if self.normalize_before:
            return self.forward_pre(src, src_mask, src_key_padding_mask, pos)
        return self.forward_post(src, src_mask, src_key_padding_mask, pos)


class TransformerDecoderLayer(nn.Module):

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 activation="relu", normalize_before=False):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self.activation = _get_activation_fn(activation)
        self.normalize_before = normalize_before

    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    def forward_post(self, tgt, memory,
                     tgt_mask: Optional[Tensor] = None,
                     memory_mask: Optional[Tensor] = None,
                     tgt_key_padding_mask: Optional[Tensor] = None,
                     memory_key_padding_mask: Optional[Tensor] = None,
                     pos: Optional[Tensor] = None,
                     query_pos: Optional[Tensor] = None):
        q = k = self.with_pos_embed(tgt, query_pos)
        tgt2 = self.self_attn(q, k, value=tgt, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        tgt2 = self.multihead_attn(query=self.with_pos_embed(tgt, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt

    def forward_pre(self, tgt, memory,
                    tgt_mask: Optional[Tensor] = None,
                    memory_mask: Optional[Tensor] = None,
                    tgt_key_padding_mask: Optional[Tensor] = None,
                    memory_key_padding_mask: Optional[Tensor] = None,
                    pos: Optional[Tensor] = None,
                    query_pos: Optional[Tensor] = None):
        tgt2 = self.norm1(tgt)
        q = k = self.with_pos_embed(tgt2, query_pos)
        tgt2 = self.self_attn(q, k, value=tgt2, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)[0]
        tgt = tgt + self.dropout1(tgt2)
        tgt2 = self.norm2(tgt)
        tgt2 = self.multihead_attn(query=self.with_pos_embed(tgt2, query_pos),
                                   key=self.with_pos_embed(memory, pos),
                                   value=memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)[0]
        tgt = tgt + self.dropout2(tgt2)
        tgt2 = self.norm3(tgt)
        tgt2 = self.linear2(self.dropout(self.activation(self.linear1(tgt2))))
        tgt = tgt + self.dropout3(tgt2)
        return tgt

    def forward(self, tgt, memory,
                tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None,
                pos: Optional[Tensor] = None,
                query_pos: Optional[Tensor] = None):
        if self.normalize_before:
            return self.forward_pre(tgt, memory, tgt_mask, memory_mask,
                                    tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos)
        return self.forward_post(tgt, memory, tgt_mask, memory_mask,
                                 tgt_key_padding_mask, memory_key_padding_mask, pos, query_pos)


def _get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])


def build_transformer(args):
    return Transformer(
        d_model=args.hidden_dim,
        dropout=args.dropout,
        nhead=args.nheads,
        dim_feedforward=args.dim_feedforward,
        num_encoder_layers=args.enc_layers,
        num_decoder_layers=args.dec_layers,
        normalize_before=args.pre_norm,
        return_intermediate_dec=True,
    )


def _get_activation_fn(activation):

    if activation == "relu":
        return F.relu
    if activation == "gelu":
        return F.gelu
    if activation == "glu":
        return F.glu
    raise RuntimeError(F"activation should be relu/gelu, not {activation}.")

## Positional Encoding

In [ ]:
import math
import torch
from torch import nn

class NestedTensor(object):
    def __init__(self, tensors, mask: Optional[Tensor]):
        self.tensors = tensors
        self.mask = mask

    def to(self, device):
        cast_tensor = self.tensors.to(device)
        mask = self.mask
        if mask is not None:
            assert mask is not None
            cast_mask = mask.to(device)
        else:
            cast_mask = None
        return NestedTensor(cast_tensor, cast_mask)

    def decompose(self):
        return self.tensors, self.mask

    def __repr__(self):
        return str(self.tensors)

class PositionEmbeddingLearned(nn.Module):

    def __init__(self, num_pos_feats=256):
        super().__init__()
        self.row_embed = nn.Embedding(50, num_pos_feats)
        self.col_embed = nn.Embedding(50, num_pos_feats)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.uniform_(self.row_embed.weight)
        nn.init.uniform_(self.col_embed.weight)

    def forward(self, x):    # tensor_list: NestedTensor
        # x = tensor_list.tensors
        h, w = x.shape[-2:]
        i = torch.arange(w, device=x.device)
        j = torch.arange(h, device=x.device)
        x_emb = self.col_embed(i)
        y_emb = self.row_embed(j)
        pos = torch.cat([
            x_emb.unsqueeze(0).repeat(h, 1, 1),
            y_emb.unsqueeze(1).repeat(1, w, 1),
        ], dim=-1).permute(2, 0, 1).unsqueeze(0).repeat(x.shape[0], 1, 1, 1)
        return pos


## Model Training

In [ ]:
query_embed=nn.Embedding(num_embeddings=1,embedding_dim=3)
x_train,y_train,x_test,y_test=Data["vertex_train"],Data["label_train"],Data["vertex_test"],Data["label_test"]
position_embedding = PositionEmbeddingLearned()
model=Transformer( d_model=3, nhead=1, num_encoder_layers=6,
                 num_decoder_layers = 6, dim_feedforward = 3, dropout = 0.1,
                 activation = "relu")
param_dict=model.parameters()
optimizer=torch.optim.AdamW(param_dict,lr=0.001,weight_decay=0.0004)
loss_fn=nn.CrossEntropyLoss()

In [ ]:
for epoch in range(1):
    for x,y in zip(x_train,y_train):
        optimizer.zero_grad()
        x=torch.tensor(x[:3])
        # print(x.shape)
        x=x.view(1,x.shape[0],x.shape[1])
        # print(x)
        # positinal_encoding=position_embedding(x)
        positinal_encoding=None
        pred=model(src=x, mask=None, query_embed=query_embed.weight.unsqueeze(1), pos_embed=positinal_encoding)
        # pred=model(src=x, mask=None, query_embed=None, pos_embed=positinal_encoding)
        loss=loss_fn(pred,y)
        loss.backward()
        optimizer.step()

In [ ]:
p=PositionEmbeddingLearned(3)
a=torch.tensor([[[3,2],[5,4],[9,8]]])
k=p(a)
k.shape

torch.Size([1, 6, 3, 2])

In [ ]:
query_embed.weight

Parameter containing:
tensor([[3.7677e-04, 6.3835e-01, 7.7134e-01]], requires_grad=True)

## Graph Attention Network (GAN) Hard Coded

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(2020)
class GATLayer(nn.Module):
  
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GATLayer, self).__init__()
        self.dropout       = dropout        # drop prob = 0.6
        self.in_features   = in_features    
        self.out_features  = out_features   
        self.alpha         = alpha          # LeakyReLU with negative input slope, alpha = 0.2
        self.concat        = concat         
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        
        
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, input, adj):
        # Linear Transformation
        h = torch.mm(input, self.W)
        N = h.size()[0]

        # Attention Mechanism
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e       = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        # Masked Attention
        zero_vec  = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime   = torch.matmul(attention, h)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime



## Graph Attention Network (Direct Code)

In [ ]:
from torch_geometric.nn import GATConv
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

## Euclidean data to Graphical Data conversion

In [ ]:
torch.__version__

'1.10.0+cu111'

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${'1.10.0+cu111'}+${'11.1'}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${'1.10.0+cu111'}+${'11.1'}.html
!pip install torch-geometric

In [ ]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

import matplotlib.pyplot as plt
%matplotlib notebook

import warnings
warnings.filterwarnings("ignore")

name_data = 'Cora'
dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)
dataset.transform = T.NormalizeFeatures()

print(f"Number of Classes in {name_data}:", dataset.num_classes)
print(f"Number of Node Features in {name_data}:", dataset.num_node_features)


## Convert Euclidean data to Graphical Data

In [ ]:
import torch
import pickle
with open("/content/drive/MyDrive/Colab Notebooks/Deep Learning/GNN/modelnet10.pickle",'rb') as data:
    Data=pickle.load(data)
x_train,y_train,x_test,y_test=Data["vertex_train"],Data["label_train"],Data["vertex_test"],Data["label_test"]
dist_thres=1.1  # hyperparameter
edge_index,Adjacency_matrix=[],[]
for obj in x_train:
    N=len(obj)
    obj=torch.tensor(obj)
    edges=[[],[]]
    adj=torch.ones(N,N)
    for i in range(N):
        for j in range(N):
            if torch.norm((obj[i]-obj[j]))<dist_thres:
                edges[0].append(i)
                edges[1].append(j)
            else:
                adj[i][j]=0
    edge_index.append(torch.tensor(edges))
    Adjacency_matrix.append(adj)
    break

edge_index1,Adjacency_matrix1=[],[]
for obj in x_test:
    N=len(obj)
    obj=torch.tensor(obj)
    edges=[[],[]]
    adj=torch.ones(N,N)
    for i in range(N):
        for j in range(N):
            if torch.norm((obj[i]-obj[j]))<dist_thres:
                edges[0].append(i)
                edges[1].append(j)
            else:
                adj[i][j]=0
    edge_index1.append(torch.tensor(edges))
    Adjacency_matrix1.append(adj)
    break
# graph_data=Data(x=x_train,y=y_train,edge_index=edge_index)
                

In [ ]:
len(edge_index1[0][0])

50681

In [ ]:
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Architecture 1

In [ ]:
class architecture1(torch.nn.Module):
    def __init__(self,input_dim=3,embed_dim=32):
        super().__init__()
        self.embed_dim=embed_dim
        self.input_dim=input_dim
        self.embed_layer=torch.nn.Linear(self.input_dim,self.embed_dim)
        self.object_query=torch.nn.Embedding(1,self.embed_dim)
        # Encoder Layer
        self.enc_attn_layer1=torch.nn.Linear(2*self.embed_dim,1)
        self.leaky_relu=torch.nn.LeakyReLU(negative_slope=0.2)
        self.norm_softmax1=torch.nn.Softmax(dim=1)
        self.enc_layer1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.Norm=torch.nn.LayerNorm(self.embed_dim)
        self.relu=torch.nn.ReLU()
        self.enc_layer2_1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        # Decoder Layer


    def forward(self,x,adjacency_matrix,edge_index):
        total_edges=len(edge_index[0])
        
        # Embedding
        input=self.embed_layer(x)

        # Encoder Layer
        enc_attn_matrix1=torch.zeros(adjacency_matrix.shape)
        for i in range(total_edges):
            node1,node2=edge_index[0][i],edge_index[1][i]
            node_pair=torch.cat((input[node1],input[node2]))
            enc_attn_matrix1[node1][node2]=self.leaky_relu1(self.attn_layer1(node_pair))
        enc_attn_matrix1=self.norm_softmax1(enc_attn_matrix1)
        attn_out=self.Norm1_1(input+torch.matmul(enc_attn_matrix1,self.relu(self.layer1_1(input))))
        enc_out_1=self.Norm2_1(attn_out+self.relu(self.layer2_1(input)))


## Architecture 2

In [ ]:
class architecture2(torch.nn.Module):
    def __init__(self,inp_dim=3,embed_dim=32,n_classes=4):
        super().__init__()
        self.n_classes=n_classes
        self.embed_dim=embed_dim
        self.inp_dim=inp_dim
        self.embed_layer=torch.nn.Linear(self.inp_dim,self.embed_dim)
        self.object_query=torch.nn.Embedding(1,self.embed_dim)

        self.softmax=torch.nn.Softmax(dim=1)
        self.relu=torch.nn.ReLU()
        self.Norm=torch.nn.LayerNorm(self.embed_dim)

        # Encoder Layer
        self.enc_key1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.enc_query1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.enc_value1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.enc_FFN1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        # Decoder Layer
        self.dec_key1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_query1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_value1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        self.dec_key2_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_value2_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_FFN1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        # Final Output Layer
        self.out=torch.nn.Linear(self.embed_dim,self.n_classes)

    def forward(self,x,adj):
        inp=self.embed_layer(x)

        # Encoder Layer
        key=torch.matmul(adj,self.enc_key1(inp))
        query=torch.matmul(adj,self.enc_query1(inp))
        value=torch.matmul(adj,self.enc_value1(inp))
        attn=self.Norm(torch.matmul(self.softmax(torch.matmul(query,torch.t(key))),value)+inp)
        enc_out=self.Norm(torch.matmul(adj,self.enc_FFN1(attn))+attn)

        # Decoder Layer
        key=self.dec_key1_1(self.object_query.weight)
        query=self.dec_query1_1(self.object_query.weight)
        value=self.dec_value1_1(self.object_query.weight)
        Q=self.Norm(torch.matmul(self.softmax(torch.matmul(query,torch.t(key))),value)+self.object_query.weight)
        key=torch.matmul(adj,self.dec_key2_1(enc_out))
        value=torch.matmul(adj,self.dec_value2_1(enc_out))
        attn=self.Norm(torch.matmul(self.softmax(torch.matmul(Q,torch.t(key))),value)+Q)
        dec_out=self.Norm(self.dec_FFN1(attn)+attn)

        # Final Output Layer
        output=torch.nn.functional.softmax(self.out(dec_out))

        return output


In [ ]:
model=architecture2().to(device)
opt=torch.optim.Adam(model.parameters(),lr=0.01)
loss_fn=torch.nn.CrossEntropyLoss()
for epoch in range(25):
    for x,y,adj in zip(x_train,y_train,Adjacency_matrix):
        x,y,adj=torch.tensor(x).to(device),torch.tensor([y],dtype=torch.int64).to(device),adj.to(device)
        opt.zero_grad()
        pred=model(x,adj)
        loss=loss_fn(pred,y)
        loss.backward()
        opt.step()
        break


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
output=[]
for x,y,adj in zip(x_test,y_test,Adjacency_matrix1):
    x,y,adj=torch.tensor(x).to(device),torch.tensor([y],dtype=torch.int64).to(device),adj.to(device)
    pred=model(x,adj)
    output.append(torch.argmax(pred))

In [ ]:
import torch
import pickle
from numpy.random import randint
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
class architecture2(torch.nn.Module):
    def __init__(self,inp_dim=3,embed_dim=32,n_classes=4):
        super().__init__()
        self.n_classes=n_classes
        self.embed_dim=embed_dim
        self.inp_dim=inp_dim
        self.embed_layer=torch.nn.Linear(self.inp_dim,self.embed_dim)
        self.object_query=torch.nn.Embedding(1,self.embed_dim)

        self.softmax=torch.nn.Softmax(dim=1)
        self.relu=torch.nn.ReLU()
        self.Norm=torch.nn.LayerNorm(self.embed_dim)

        # Encoder Layer
        self.enc_key1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.enc_query1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.enc_value1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.enc_FFN1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        # Decoder Layer
        self.dec_key1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_query1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_value1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        self.dec_key2_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_value2_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_FFN1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        # Final Output Layer
        self.out=torch.nn.Linear(self.embed_dim,self.n_classes)

    def forward(self,x):
        inp=self.embed_layer(x)

        # Encoder Layer
        key=self.enc_key1(inp)
        query=self.enc_query1(inp)
        value=self.enc_value1(inp)
        attn=self.Norm(torch.matmul(self.softmax(torch.matmul(query,torch.t(key))),value)+inp)
        enc_out=self.Norm(self.enc_FFN1(attn)+attn)

        # Decoder Layer
        key=self.dec_key1_1(self.object_query.weight)
        query=self.dec_query1_1(self.object_query.weight)
        value=self.dec_value1_1(self.object_query.weight)
        Q=self.Norm(torch.matmul(self.softmax(torch.matmul(query,torch.t(key))),value)+self.object_query.weight)
        key=self.dec_key2_1(enc_out)
        value=self.dec_value2_1(enc_out)
        attn=self.Norm(torch.matmul(self.softmax(torch.matmul(Q,torch.t(key))),value)+Q)
        dec_out=self.Norm(self.dec_FFN1(attn)+attn)

        # Final Output Layer
        output=torch.nn.functional.softmax(self.out(dec_out))

        return output


model=architecture2().to(device)
opt=torch.optim.Adam(model.parameters(),lr=0.01)
loss_fn=torch.nn.CrossEntropyLoss()
with open("/content/drive/MyDrive/Colab Notebooks/Deep Learning/GNN/modelnet10.pickle",'rb') as data:
    Data=pickle.load(data)
x_train,y_train,x_test,y_test=Data["vertex_train"],Data["label_train"],Data["vertex_test"],Data["label_test"]
dist_thres=1.1

In [ ]:
max_size=5000
for epoch in range(100000):
    # for x,y in zip(x_train,y_train):
    idx=randint(0,len(x_train))
        # x,y=torch.tensor(x).to(device),torch.tensor([y],dtype=torch.int64).to(device)
    x,y=torch.tensor(x_train[idx]).to(device),torch.tensor([y_train[idx]],dtype=torch.int64).to(device)
    if len(x)<max_size:
        opt.zero_grad()
        pred=model(x)
        loss=loss_fn(pred,y)
        loss.backward()
        opt.step()

correct,total=0,0
for epoch in range(250):
    idx=randint(0,len(x_test))
    x=torch.tensor(x_test[idx]).to(device)
    if len(x)<max_size:
        pred=torch.argmax(model(x))
        total+=1
        if y_test[idx]==pred:
            correct+=1
accuracy=correct*100/total
print("Accuracy: ",accuracy)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy:  30.386740331491712


In [1]:
import torch
import pickle
from numpy.random import randint
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

class architecture2(torch.nn.Module):
    def __init__(self,inp_dim=3,embed_dim=64,n_classes=4):
        super().__init__()
        self.n_classes=n_classes
        self.embed_dim=embed_dim
        self.inp_dim=inp_dim
        self.embed_layer=torch.nn.Linear(self.inp_dim,self.embed_dim)
        self.object_query=torch.nn.Embedding(1,self.embed_dim)

        self.softmax=torch.nn.Softmax(dim=1)
        self.relu=torch.nn.ReLU()
        self.Norm=torch.nn.LayerNorm(self.embed_dim)

        # Encoder Layer
        self.enc_key1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.enc_query1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.enc_value1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.enc_FFN1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        # Decoder Layer
        self.dec_key1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_query1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_value1_1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        self.dec_key2_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_value2_1=torch.nn.Linear(self.embed_dim,self.embed_dim)
        self.dec_FFN1=torch.nn.Linear(self.embed_dim,self.embed_dim)

        # Final Output Layer
        self.out=torch.nn.Linear(self.embed_dim,self.n_classes)

    def forward(self,x,adj):
        inp=self.embed_layer(x)

        # Encoder Layer
        key=torch.matmul(adj,self.enc_key1(inp))
        query=torch.matmul(adj,self.enc_query1(inp))
        value=torch.matmul(adj,self.enc_value1(inp))
        attn=self.Norm(torch.matmul(self.softmax(torch.matmul(query,torch.t(key))),value)+inp)
        enc_out=self.Norm(torch.matmul(adj,self.enc_FFN1(attn))+attn)

        # Decoder Layer
        key=self.dec_key1_1(self.object_query.weight)
        query=self.dec_query1_1(self.object_query.weight)
        value=self.dec_value1_1(self.object_query.weight)
        Q=self.Norm(torch.matmul(self.softmax(torch.matmul(query,torch.t(key))),value)+self.object_query.weight)
        key=torch.matmul(adj,self.dec_key2_1(enc_out))
        value=torch.matmul(adj,self.dec_value2_1(enc_out))
        attn=self.Norm(torch.matmul(self.softmax(torch.matmul(Q,torch.t(key))),value)+Q)
        dec_out=self.Norm(self.dec_FFN1(attn)+attn)

        # Final Output Layer
        output=torch.nn.functional.softmax(self.out(dec_out))

        return output

In [2]:
model=architecture2().to(device)
opt=torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=0.007)
loss_fn=torch.nn.CrossEntropyLoss()
with open("/content/drive/MyDrive/Colab Notebooks/Deep Learning/GNN/modelnet10.pickle",'rb') as data:
    Data=pickle.load(data)
x_train,y_train=Data["vertex_train"],Data["label_train"]
with open("/content/drive/MyDrive/Colab Notebooks/Deep Learning/GNN/test_data.pickle",'rb') as data:
    Data=pickle.load(data)
x_test,y_test=Data["vertex_test"],Data["label_test"]
dist_thres=0.5
min_points=1000

In [3]:
for epoch in range(3000):
    # for x,y in zip(x_train,y_train):
    idx=randint(0,len(x_train))
        # x,y=torch.tensor(x).to(device),torch.tensor([y],dtype=torch.int64).to(device)
    x,y=torch.tensor(x_train[idx]).to(device),torch.tensor([y_train[idx]],dtype=torch.int64).to(device)
    N=len(x)
    if N<min_points:
        adj=torch.ones(N,N).to(device)
        for i in range(N):
            for j in range(i+1,N):
                if torch.norm(x[i]-x[j])>dist_thres:
                    adj[i][j]=0
                    adj[j][i]=0


        opt.zero_grad()
        pred=model(x,adj)
        loss=loss_fn(pred,y)
        loss.backward()
        opt.step()
#         print("epoch: ",epoch)
    
correct=0
total=0
for epoch in range(250):
    idx=randint(0,len(x_test))
    x=torch.tensor(x_test[idx]).to(device)
    N=len(x)
    if N<min_points:
        adj=torch.ones(N,N).to(device)
        for i in range(N):
            for j in range(i+1,N):
                if torch.norm(x[i]-x[j])>dist_thres:
                    adj[i][j]=0
                    adj[j][i]=0
        pred=torch.argmax(model(x,adj))
        total+=1
        if y_test[idx]==pred:
            correct+=1
        
accuracy=correct*100/total
print("Accuracy: ",accuracy)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy:  31.25


In [ ]:
device

device(type='cpu')